In [1]:
import numpy as np
import pickle
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

In [2]:
def filterPred(predFile, values, toRemove, top=1):
    pred = pickle.load(open(predFile, 'rb'))
    y = pred['y']
    yp = pred['yp']
    if not toRemove is None:
        y = np.delete(y, [np.argwhere(values==r)[0][0] for r in toRemove], axis=1)
        yp = np.delete(yp, [np.argwhere(values==r)[0][0] for r in toRemove], axis=1)
        
        rowsToDelete = np.argwhere(np.amax(y,1)==0)
        y = np.delete(y, rowsToDelete, axis=0)
        yp = np.delete(yp, rowsToDelete, axis=0)
          
    yb = []
    ypb = []
    
    for i in range(len(y)):
        yb.append(np.argmax(y[i]))
        ypb.append(np.argmax(yp[i]))
        
    yb = np.array(yb)
    
    if top == 1:
        ypb = np.array(ypb)
    elif top == 3:
        ypb = np.argsort(yp,1)[:,-3:][:,::-1] #top 3 
    elif top == 5:
        ypb = np.argsort(yp,1)[:,-5:][:,::-1] #top 5 
    else:
        ypb = 0
    
    return yb, ypb

In [3]:
def calculateMcNemar(preds, values, toRemove, top=1):
    d = {'yp{}'.format(i): filterPred(pred, values, toRemove, top)[1] for i,pred in enumerate(preds)}
    d['y'] = filterPred(preds[0], values, toRemove)[0] #first
    #df = pd.DataFrame(d)
    ret = np.zeros([len(preds), len(preds)])
    for i in range(len(preds)):
        for j in range(len(preds)):
            cont = np.zeros([2,2])
            for n in range(len(d['y'])):
                if top == 1:
                    vi = d['y'][n] == d['yp{}'.format(i)][n]
                    vj = d['y'][n] == d['yp{}'.format(j)][n]
                else:
                    vi = d['y'][n] in d['yp{}'.format(i)][n]
                    vj = d['y'][n] in d['yp{}'.format(j)][n]
                    
                if vi and vj:
                    cont[0,0] += 1
                elif vi and not vj:
                    cont[1,0] += 1
                elif not vi and vj:
                    cont[0,1] += 1
                else:
                    cont[1,1] += 1
                    
            if np.min(cont) < 25:
                ret[i,j] = mcnemar(cont, exact=True).pvalue
            else:
                ret[i,j] = mcnemar(cont, exact=False, correction=True).pvalue
                
            #if np.min(np.array(pd.crosstab(df['yp{}'.format(i)]==df['y'], df['yp{}'.format(j)]==df['y']))) < 25:
            #    ret[i,j] = mcnemar(pd.crosstab(df['yp{}'.format(i)]==df['y'], df['yp{}'.format(j)]==df['y']), exact=True).pvalue
            #else:
            #    ret[i,j] = mcnemar(pd.crosstab(df['yp{}'.format(i)]==df['y'], df['yp{}'.format(j)]==df['y']), exact=False, correction=True).pvalue
        
    return ret

In [4]:
toRemoveS = pickle.load(open("toRemoveSite.p", 'rb'))
toRemoveM = pickle.load(open("toRemoveMorfo.p", 'rb'))

In [5]:
valuesS = pickle.load(open('corpusTemporal/valuesTemporal.p', 'rb'))
valuesM = pickle.load(open('corpusTemporalV2b/valuesTemporalMorfo1.p', 'rb'))

In [6]:
predsS = [
        'predictionsSVM-Asite.p',
        'predictionsCNN-Asite.p',
        'predictionGRU-Asite.p',
        'predictionsBERT-Asite.p',
        'predictionMAXi-Asite.p',
        'predictionMAXh-Asite.p',
        'predictionATTh-Asite.p',
        'predictionMAX-Asite.p',
        'predictionATT-Asite.p',
    ]

predsM = [
        'predictionsSVM-Amorpho.p',
        'predictionsCNN-Amorpho.p',
        'predictionGRU-Amorpho.p',
        'predictionsBERT-Amorpho.p',
        'predictionMAXi-Amorpho.p',
        'predictionMAXh-Amorpho.p',
        'predictionATTh-Amorpho.p',
        'predictionMAX-Amorpho.p',
        'predictionATT-Amorpho.p',
    ]

## calculate examples

In [7]:
def calculateLengths(corpus, key, toRemove):
    count = {d:len([i for i in corpus[d][key] if i not in toRemove]) for d in corpus}
    print(count)

In [8]:
corpusS =  pickle.load(open('corpusTemporal/corpusTemporal.p', 'rb'))
corpusM =  pickle.load(open('corpusTemporalV2b/corpusTemporal.p', 'rb'))
keyS = 'sede1'
keyM = 'morfo1'

In [9]:
calculateLengths(corpusS, keyS, toRemoveS)

{'train': 50875, 'valid': 17007, 'test': 17015}


In [10]:
calculateLengths(corpusM, keyM, toRemoveM)

{'train': 49436, 'valid': 16787, 'test': 16719}


# Site

In [11]:
mcNemarSite1 = calculateMcNemar(predsS, valuesS, toRemoveS, 1)
mcNemarSite3 = calculateMcNemar(predsS, valuesS, toRemoveS, 3)
mcNemarSite5 = calculateMcNemar(predsS, valuesS, toRemoveS, 5)

In [22]:
with np.printoptions(precision=5, suppress=True):
    print(mcNemarSite1)

[[1.      0.00612 0.41073 0.34979 0.      0.34109 0.25423 0.00086 0.01619]
 [0.00612 1.      0.00014 0.00012 0.      0.00004 0.00002 0.      0.     ]
 [0.41073 0.00014 1.      0.9422  0.      0.91086 0.74042 0.00184 0.05873]
 [0.34979 0.00012 0.9422  1.      0.      1.      0.83493 0.0057  0.09542]
 [0.      0.      0.      0.      1.      0.      0.      0.      0.     ]
 [0.34109 0.00004 0.91086 1.      0.      1.      0.8421  0.004   0.08715]
 [0.25423 0.00002 0.74042 0.83493 0.      0.8421  1.      0.01011 0.13819]
 [0.00086 0.      0.00184 0.0057  0.      0.004   0.01011 1.      0.25339]
 [0.01619 0.      0.05873 0.09542 0.      0.08715 0.13819 0.25339 1.     ]]


In [24]:
with np.printoptions(precision=5, suppress=True):
    print(mcNemarSite3)

[[1.      0.42222 0.00003 0.00176 0.0021  0.02732 0.00499 0.      0.06035]
 [0.42222 1.      0.0001  0.01137 0.00001 0.11004 0.02327 0.      0.20617]
 [0.00003 0.0001  1.      0.18763 0.      0.01885 0.10323 0.07637 0.00779]
 [0.00176 0.01137 0.18763 1.      0.      0.36511 0.83721 0.00218 0.22247]
 [0.0021  0.00001 0.      0.      1.      0.      0.      0.      0.     ]
 [0.02732 0.11004 0.01885 0.36511 0.      1.      0.4808  0.00005 0.80355]
 [0.00499 0.02327 0.10323 0.83721 0.      0.4808  1.      0.00079 0.33846]
 [0.      0.      0.07637 0.00218 0.      0.00005 0.00079 1.      0.     ]
 [0.06035 0.20617 0.00779 0.22247 0.      0.80355 0.33846 0.      1.     ]]


In [25]:
with np.printoptions(precision=5, suppress=True):
    print(mcNemarSite5)

[[1.      0.      0.      0.      0.00001 0.      0.      0.      0.     ]
 [0.      1.      0.56818 0.3011  0.      0.1763  0.6582  0.00001 0.69466]
 [0.      0.56818 1.      0.69912 0.      0.44988 0.95381 0.00006 0.29627]
 [0.      0.3011  0.69912 1.      0.      0.78377 0.61377 0.00152 0.14799]
 [0.00001 0.      0.      0.      1.      0.      0.      0.      0.     ]
 [0.      0.1763  0.44988 0.78377 0.      1.      0.33692 0.00383 0.07229]
 [0.      0.6582  0.95381 0.61377 0.      0.33692 1.      0.0001  0.35481]
 [0.      0.00001 0.00006 0.00152 0.      0.00383 0.0001  1.      0.     ]
 [0.      0.69466 0.29627 0.14799 0.      0.07229 0.35481 0.      1.     ]]


In [15]:
print(mcNemarSite1 < 0.05) #True: different proportion errors

[[False  True False False  True False False  True  True]
 [ True False  True  True  True  True  True  True  True]
 [False  True False False  True False False  True False]
 [False  True False False  True False False  True False]
 [ True  True  True  True False  True  True  True  True]
 [False  True False False  True False False  True False]
 [False  True False False  True False False  True False]
 [ True  True  True  True  True  True  True False False]
 [ True  True False False  True False False False False]]


# Morpho

In [16]:
mcNemarMorpho1 = calculateMcNemar(predsM, valuesM, toRemoveM, 1)
mcNemarMorpho3 = calculateMcNemar(predsM, valuesM, toRemoveM, 3)
mcNemarMorpho5 = calculateMcNemar(predsM, valuesM, toRemoveM, 5)

In [23]:
with np.printoptions(precision=5, suppress=True):
    print(mcNemarMorpho1)

[[1.      0.00004 0.00026 0.      0.      0.      0.      0.      0.     ]
 [0.00004 1.      0.97881 0.00008 0.      0.07425 0.05557 0.      0.     ]
 [0.00026 0.97881 1.      0.00012 0.      0.07224 0.05303 0.      0.     ]
 [0.      0.00008 0.00012 1.      0.      0.01539 0.02884 0.17715 0.01728]
 [0.      0.      0.      0.      1.      0.      0.      0.      0.     ]
 [0.      0.07425 0.07224 0.01539 0.      1.      0.92889 0.00005 0.     ]
 [0.      0.05557 0.05303 0.02884 0.      0.92889 1.      0.00008 0.     ]
 [0.      0.      0.      0.17715 0.      0.00005 0.00008 1.      0.23144]
 [0.      0.      0.      0.01728 0.      0.      0.      0.23144 1.     ]]


In [26]:
with np.printoptions(precision=5, suppress=True):
    print(mcNemarMorpho3)

[[1.      0.01417 0.00032 0.00002 0.      0.02159 0.03758 0.      0.     ]
 [0.01417 1.      0.13729 0.      0.      0.97006 0.88142 0.00004 0.00119]
 [0.00032 0.13729 1.      0.      0.      0.17112 0.0851  0.00373 0.04428]
 [0.00002 0.      0.      1.      0.      0.      0.      0.      0.     ]
 [0.      0.      0.      0.      1.      0.      0.      0.      0.     ]
 [0.02159 0.97006 0.17112 0.      0.      1.      0.8118  0.00005 0.00141]
 [0.03758 0.88142 0.0851  0.      0.      0.8118  1.      0.00001 0.00027]
 [0.      0.00004 0.00373 0.      0.      0.00005 0.00001 1.      0.49072]
 [0.      0.00119 0.04428 0.      0.      0.00141 0.00027 0.49072 1.     ]]


In [27]:
with np.printoptions(precision=5, suppress=True):
    print(mcNemarMorpho5)

[[1.      0.      0.      0.0002  0.      0.      0.00024 0.      0.     ]
 [0.      1.      0.54089 0.      0.      0.03366 0.00041 0.08691 0.11752]
 [0.      0.54089 1.      0.      0.      0.1487  0.00213 0.00881 0.01922]
 [0.0002  0.      0.      1.      0.      0.      0.      0.      0.     ]
 [0.      0.      0.      0.      1.      0.      0.      0.      0.     ]
 [0.      0.03366 0.1487  0.      0.      1.      0.14035 0.00009 0.00023]
 [0.00024 0.00041 0.00213 0.      0.      0.14035 1.      0.      0.     ]
 [0.      0.08691 0.00881 0.      0.      0.00009 0.      1.      0.95982]
 [0.      0.11752 0.01922 0.      0.      0.00023 0.      0.95982 1.     ]]


In [20]:
print(mcNemarMorpho1 < 0.05) #True: different proportion errors

[[False  True  True  True  True  True  True  True  True]
 [ True False False  True  True False False  True  True]
 [ True False False  True  True False False  True  True]
 [ True  True  True False  True  True  True False  True]
 [ True  True  True  True False  True  True  True  True]
 [ True False False  True  True False False  True  True]
 [ True False False  True  True False False  True  True]
 [ True  True  True False  True  True  True False False]
 [ True  True  True  True  True  True  True False False]]


In [21]:
#np.savez("mcNemar.npz", site1=mcNemarSite1, site3=mcNemarSite3, site5=mcNemarSite5, morpho1=mcNemarMorpho1, morpho3=mcNemarMorpho3, morpho5=mcNemarMorpho5)